In [1]:
import sys
sys.path.append('shallow-trt')

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Imports

In [3]:
#export
import sys
import os
import logging
from itertools import chain

import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import tensorrt as trt

# Code

In [4]:
#export
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
def GiB(val): return val * 1 << 30


In [5]:
#export
def parse_onnx(onnx_file_path, explicit_batch=None, max_batch_size=1):
    TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE)
    trt.init_libnvinfer_plugins(TRT_LOGGER, '') 
    builder = trt.Builder(TRT_LOGGER)

    if explicit_batch is not None:
        EXPLICIT_BATCH = explicit_batch << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
        network = builder.create_network(EXPLICIT_BATCH)
    else:
        network = builder.create_network()
        builder.max_batch_size = max_batch_size

    parser = trt.OnnxParser(network, TRT_LOGGER)    
    logger = logging.getLogger('onnx_parser')

    with open(onnx_file_path, 'rb') as model:
        if not parser.parse(model.read()):
            for error in range(parser.num_errors):
                logger.debug(parser.get_error(error))

    logger.debug(f'NETWORK PARSE: {len(network)}')
    return builder, network, TRT_LOGGER

def build_trt_engine(builder, network, mode='fp32', max_workspace_size_gb=1):
    builder.fp16_mode = True if mode == 'fp16' else False
    builder.int8_mode = True if mode == 'int8' else False
    
    builder.max_workspace_size = GiB(max_workspace_size_gb)
    engine = builder.build_cuda_engine(network)
    return engine
    
def save_engine(engine, engine_dest_path):
    buf = engine.serialize()
    with open(engine_dest_path, 'wb') as f:
        f.write(buf)

def load_engine(trt_runtime, engine_path):
    with open(engine_path, 'rb') as f:
        engine_data = f.read()
    engine = trt_runtime.deserialize_cuda_engine(engine_data)
    return engine

def init_trt():
    trt_logger = trt.Logger(trt.Logger.INFO)
    trt.init_libnvinfer_plugins(trt_logger, '')
    runtime = trt.Runtime(trt_logger)
    return trt_logger, runtime

In [ ]:
#export
class HostDeviceMem:
    def __init__(self, host_mem, device_mem):
        self.h = host_mem
        self.d = device_mem

    def __str__(self):   return "Host:\n" + str(self.h) + "\nDevice:\n" + str(self.d)
    def __repr__(self):  return self.__str__()
    
    def htod(self, stream):
        cuda.memcpy_htod_async(self.d, self.h, stream)
        #stream.synchronize()
        
    def dtoh(self, stream):
        cuda.memcpy_dtoh_async(self.h, self.d, stream)
        #stream.synchronize()
        
def allocate_buffers(engine, dtypes=None):
    """
    Args:
        engine (trt.ICudaEngine): TensorRT engine
    Returns:
        inputs [HostDeviceMem]: engine input memory
        outputs [HostDeviceMem]: engine output memory
        bindings [int]: buffer to device bindings
        stream (cuda.Stream): cuda stream for engine inference synchronization
    """
    inputs = []
    outputs = []
    bindings = []
    stream = cuda.Stream()
    for i, binding in enumerate(engine):
        size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
        if dtypes is not None: 
            dtype = dtypes[i]
        else:
            dtype = trt.nptype(engine.get_binding_dtype(binding))
        
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        bindings.append(int(device_mem))
        
        if engine.binding_is_input(binding):
            inputs.append(HostDeviceMem(host_mem, device_mem))
        else:
            outputs.append(HostDeviceMem(host_mem, device_mem))
    return inputs, outputs, bindings, stream


def do_inference(context, bindings, inputs, outputs, stream, batch_size=1, explicit_batch=False):
    [inp.htod(stream) for inp in inputs]
    context_exec = context.execute_async_v2 if explicit_batch else context.execute_async
    context_exec(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    [out.dtoh(stream) for out in outputs]
    stream.synchronize()
    return [out.h for out in outputs]

In [6]:
#export
class BaseEngine:
    """Base engine class
        Creates TRT engine, allocates memory
    """    

    def __init__(self, config):
        """
        config: Dict
        {
            'engine':'path/to/engine.trt',
   optional 'dtypes':[trt.DataType.BOOL, trt.DataType.FLOAT, trt.DataType.HALF, trt.DataType.INT32, trt.DataType.INT8]
   
       }
        """
        self.config = config
        self.trt_logger, self.runtime = init_trt()
        self.engine = load_engine(self.runtime, self.config['engine'])
        self.context = self.engine.create_execution_context()
        dtypes = self.config.get('dtypes', None)
        self.inputs, self.outputs, self.bindings, self.stream = allocate_buffers(self.engine, dtypes=dtypes)
    
    def sync(self): self.stream.synchronize()

    def info(self):
        print([self.engine.get_binding_name(i) for i in range(self.engine.num_bindings)])
        print([self.engine.get_binding_shape(i) for i in range(self.engine.num_bindings)])
        print([self.engine.get_binding_dtype(i) for i in range(self.engine.num_bindings)])
        
    def execute(self, bindings, sync=True):
        self.context.execute_async(bindings=bindings, stream_handle=self.stream.handle)
        if sync: self.sync()
    
    def load_input(self, inp):
        """
        inp is 4D tensor or list of 3D tensors
        """
        #assert isinstance(inp, list), 'input must be a list'
        
        for i, inputs in zip(inp, self.inputs):
            np.copyto(inputs.h, i.ravel())
    
    def dtoh(self, sync=False):
        [out.dtoh(self.stream) for out in self.outputs]
        if sync: self.sync()
    
    def htod(self, sync=False):
        [inp.htod(self.stream) for inp in self.inputs]
        if sync: self.sync()
        
    def _start(self):
        self.dtoh(sync=True)
        self.execute(self.bindings)
        self.htod(sync=True)
        return [out.h for out in self.outputs]

# Tests

In [ ]:
import cv2
import time

def prepare_image(img_in, size):
    img = cv2.resize(img_in,size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.transpose(2, 0, 1) # to CHW
    return img/128 - 1 

def load_test_image(path, size):
    img = cv2.imread(path)
    img_prep = prepare_image(img, size)
    return img_prep

In [ ]:
cfg = {
    'engine':'/tmp/test.trt'
}

In [ ]:
e = BaseEngine(cfg)

In [ ]:
e._start()